In [ ]:

import pandas as pd
import numpy as np
from datetime import datetime, date
from io import BytesIO
from ipywidgets import (
    FileUpload, Button, VBox, Output, Label, SelectMultiple, HBox,
    FloatSlider, Dropdown, Text
)
from IPython.display import display, clear_output
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import objective_functions
import copy

output_all = Output()
df_global = None
selected_assets = []
sector_mapper = {}
sector_constraint_widgets = {}
expected_return_inputs = {}
constraint_rows = []
constraint_inputs = []

upload = FileUpload(accept=".xlsx", multiple=False)

def log_returns(df):
    return np.log(df / df.shift(1)).dropna()

def ewma_covariance_matrix(X, alpha):
    T, D = X.shape
    cov = np.zeros((D, D))
    for tt in range(T):
        x_t = X[tt, :]
        if tt == 0:
            cov = x_t[:, np.newaxis] @ x_t[np.newaxis, :]
        else:
            cov = alpha * cov + (1 - alpha) * x_t[:, np.newaxis] @ x_t[np.newaxis, :]
    return cov

# Constraint binding helpers
def make_equal_constraint(i, v): return lambda x: x[i] == v
def make_le_constraint(i, v): return lambda x: x[i] <= v
def make_ge_constraint(i, v): return lambda x: x[i] >= v
def make_equal_pair_constraint(i, j): return lambda x: x[i] == x[j]
def make_le_pair_constraint(i, j): return lambda x: x[i] <= x[j]
def make_ge_pair_constraint(i, j): return lambda x: x[i] >= x[j]

def show_asset_selector():
    with output_all:
        clear_output()
        asset_select = SelectMultiple(description="Assets:", options=df_global.columns.tolist(), rows=10)
        continue_btn = Button(description="➡ Define Sectors", button_style="primary")

        def proceed(change):
            global selected_assets
            selected_assets = list(asset_select.value)
            if not selected_assets:
                print("⚠️ Please select at least one asset.")
                return
            show_expected_return_inputs()

        continue_btn.on_click(proceed)
        display(VBox([
            Label("📊 Step 2: Select assets:"),
            asset_select,
            continue_btn
        ]))

def show_expected_return_inputs():
    with output_all:
        clear_output()
        global expected_return_inputs
        expected_return_inputs = {}
        rows = []

        for name in selected_assets:
            label = Label(value=name, layout={'width': '200px'})
            er_input = Text(placeholder='Expected Return', layout={'width': '100px'})
            expected_return_inputs[name] = er_input
            rows.append(HBox([label, er_input]))

        next_btn = Button(description="➡ Define Sectors", button_style="primary")
        next_btn.on_click(lambda change: show_sector_mapping())
        display(VBox([
            Label("📈 Step 3: Enter expected returns (e.g. 0.05 = 5%):"),
            VBox(rows),
            next_btn
        ]))

def show_sector_mapping():
    with output_all:
        clear_output()
        mapping_widgets = {}
        rows = []

        for asset in selected_assets:
            label = Label(value=asset, layout={"width": "200px"})
            dropdown = Dropdown(
                options=["Equities", "Corporate Bonds", "High Yield", "Euro Government Core", "Cash"],
                layout={"width": "200px"}
            )
            mapping_widgets[asset] = dropdown
            rows.append(HBox([label, dropdown]))

        next_btn = Button(description="➡ Set Sector Constraints", button_style="primary")

        def proceed_mapping(change):
            global sector_mapper
            sector_mapper = {a: w.value for a, w in mapping_widgets.items()}
            show_sector_constraints()

        next_btn.on_click(proceed_mapping)
        display(VBox([
            Label("🏷 Step 4: Assign sectors to assets:"),
            VBox(rows),
            next_btn
        ]))

def show_additional_constraints():
    with output_all:
        for row in constraint_rows:
            row.close()
        constraint_rows.clear()
        constraint_inputs.clear()

        for _ in range(3):  # up to 3 constraints
            asset1_dd = Dropdown(options=selected_assets, description="Asset A")
            relation_dd = Dropdown(options=["==", "<=", ">="], description="Relation")
            asset2_dd = Dropdown(options=selected_assets + ["Value"], description="Asset B/Value")
            value_input = Text(placeholder="0.1")

            constraint_inputs.append((asset1_dd, relation_dd, asset2_dd, value_input))
            row = HBox([asset1_dd, relation_dd, asset2_dd, value_input])
            constraint_rows.append(row)

        display(VBox([Label("🧩 Additional Constraints (A <= B or A == 0.1):")] + constraint_rows))

def apply_additional_constraints(ef, var_names):
    for asset1_dd, relation_dd, asset2_dd, value_input in constraint_inputs:
        a1 = asset1_dd.value
        rel = relation_dd.value
        a2 = asset2_dd.value
        if not a1 or not a2:
            continue

        idx1 = var_names.index(a1)

        if a2 == "Value":
            try:
                val = float(value_input.value)
            except:
                continue

            if rel == "==":
                ef.add_constraint(make_equal_constraint(idx1, val))
            elif rel == "<=":
                ef.add_constraint(make_le_constraint(idx1, val))
            elif rel == ">=":
                ef.add_constraint(make_ge_constraint(idx1, val))
        else:
            try:
                idx2 = var_names.index(a2)
                if rel == "==":
                    ef.add_constraint(make_equal_pair_constraint(idx1, idx2))
                elif rel == "<=":
                    ef.add_constraint(make_le_pair_constraint(idx1, idx2))
                elif rel == ">=":
                    ef.add_constraint(make_ge_pair_constraint(idx1, idx2))
            except:
                continue
